---
title: "Constraint Satisfaction Problems (CSP) - OR Tools (Part1)"
format: html
page-layout: full
code-line-numbers: true
code-block-border: true
toc: true
toc-location: left
number-sections: true
jupyter: python3
---

 - https://developers.google.com/optimization
   - pip install ortools

In [1]:
from ortools.sat.python import cp_model

# CP-SAT Solver

- Three variables x, y, z (each with domain [0,2])
- All the variables should have different values

In [2]:
def SimpleSatProgram(lower=0, upper=2):
    model = cp_model.CpModel()

    # Create the variables
    x = model.NewIntVar(lower, upper, "x")
    y = model.NewIntVar(lower, upper, "y")
    z = model.NewIntVar(lower, upper, "z")

    # Create the constraints
    model.Add(x != y)
    model.Add(y != z)
    model.Add(x != z)

    # Create the solver and solve the constraints
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Look for solutions
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"x, y, z = {solver.Value(x)}, {solver.Value(y)}, {solver.Value(z)}")
    else:
        print("No solutions...")

In [3]:
SimpleSatProgram()

x, y, z = 0, 1, 2


In [4]:
SimpleSatProgram()

x, y, z = 2, 1, 0


## Finding all solutions

In [5]:
# Callback for solution printer

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        print(f"{self.__solution_count:>2d}) ", end="")
        for v in self.__variables:
            print(f"{v}={self.Value(v)}", end=" ")
        print()

    def solution_count(self):
        return self.__solution_count

In [6]:
def SimpleSatProgramAllSolutions(lower=0, upper=2):
    model = cp_model.CpModel()

    # Create the variables
    x = model.NewIntVar(lower, upper, "x")
    y = model.NewIntVar(lower, upper, "y")
    z = model.NewIntVar(lower, upper, "z")

    # Create the constraints
    model.Add(x != y)
    model.Add(y != z)
    model.Add(x != z)

    # Create the solver and solve the constraints
    solver = cp_model.CpSolver()
    solver.parameters.enumerate_all_solutions = True

    solution_printer = VarArraySolutionPrinter([x, y, z])
    status = solver.Solve(model, solution_printer)

    print(f"Number of solutions found: {solution_printer.solution_count()}")


In [7]:
SimpleSatProgramAllSolutions(1,4)

 1) x=3 y=2 z=1 
 2) x=3 y=1 z=2 
 3) x=2 y=1 z=3 
 4) x=4 y=1 z=2 
 5) x=4 y=1 z=3 
 6) x=4 y=2 z=3 
 7) x=4 y=2 z=1 
 8) x=4 y=3 z=1 
 9) x=4 y=3 z=2 
10) x=3 y=2 z=4 
11) x=3 y=1 z=4 
12) x=2 y=1 z=4 
13) x=1 y=2 z=4 
14) x=1 y=3 z=4 
15) x=2 y=3 z=4 
16) x=1 y=2 z=3 
17) x=1 y=4 z=3 
18) x=2 y=4 z=3 
19) x=2 y=3 z=1 
20) x=1 y=3 z=2 
21) x=1 y=4 z=2 
22) x=2 y=4 z=1 
23) x=3 y=4 z=1 
24) x=3 y=4 z=2 
Number of solutions found: 24


# Optimization Problems

- Maximize 2x + 2y + 3z subject to the following constraints:
    - 2x + 7y + 3z <= 50
    - 3x - 5y + 7z <= 45
    - 5x + 2y - 6z <= 37
    - x, y, z >= 0
    - x, y, z are integers 

In [8]:
def OptimizationProblemExample():
    
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    var_upper_bound = max(50, 45, 37)
    
    x = model.NewIntVar(0, var_upper_bound, "x")
    y = model.NewIntVar(0, var_upper_bound, "y")
    z = model.NewIntVar(0, var_upper_bound, "z")

    # Creates the constraints.
    model.Add(2 * x + 7 * y + 3 * z <= 50)
    model.Add(3 * x - 5 * y + 7 * z <= 45)
    model.Add(5 * x + 2 * y - 6 * z <= 37)

    model.Maximize(2 * x + 2 * y + 3 * z)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Maximum of objective function: {solver.ObjectiveValue()}\n")
        print(f"x, y, z = {solver.Value(x)}, {solver.Value(y)}, {solver.Value(z)}")
    else:
        print("No solution found.")

    # Statistics.
    print("\nStatistics")
    print(f"  status   : {solver.StatusName(status)}")
    print(f"  conflicts: {solver.NumConflicts()}")
    print(f"  branches : {solver.NumBranches()}")
    print(f"  wall time: {solver.WallTime()} s")

In [9]:
OptimizationProblemExample()

Maximum of objective function: 35.0

x, y, z = 7, 3, 5

Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 0
  wall time: 0.005089000000000001 s


# SEND + MORE = MONEY

In [10]:
def send_more_money():
    """Solve the cryptarithmic puzzle SEND+MORE=MONEY."""
    model = cp_model.CpModel()

    # Create variables.
    # Since s is a leading digit, it can't be 0.
    s = model.NewIntVar(1, 9, "s")
    e = model.NewIntVar(0, 9, "e")
    n = model.NewIntVar(0, 9, "n")
    d = model.NewIntVar(0, 9, "d")
    # Since m is a leading digit, it can't be 0.
    m = model.NewIntVar(1, 9, "m")
    o = model.NewIntVar(0, 9, "o")
    r = model.NewIntVar(0, 9, "r")
    y = model.NewIntVar(0, 9, "y")

    # Create carry variables. c0 is true if the first column of addends carries
    # a 1, c2 is true if the second column carries a 1, and so on.
    c0 = model.NewBoolVar("c0")
    c1 = model.NewBoolVar("c1")
    c2 = model.NewBoolVar("c2")
    c3 = model.NewBoolVar("c3")

    # Force all letters to take on different values.
    model.AddAllDifferent(s, e, n, d, m, o, r, y)

    # Column 0:
    model.Add(c0 == m)

    # Column 1:
    model.Add(c1 + s + m == o + 10 * c0)

    # Column 2:
    model.Add(c2 + e + o == n + 10 * c1)

    # Column 3:
    model.Add(c3 + n + r == e + 10 * c2)

    # Column 4:
    model.Add(d + e == y + 10 * c3)

    # Solve model.
    solver = cp_model.CpSolver()
    if solver.Solve(model) == cp_model.OPTIMAL:
        print("Optimal solution found!")
    print("s:", solver.Value(s))
    print("e:", solver.Value(e))
    print("n:", solver.Value(n))
    print("d:", solver.Value(d))
    print("m:", solver.Value(m))
    print("o:", solver.Value(o))
    print("r:", solver.Value(r))
    print("y:", solver.Value(y))

send_more_money()

Optimal solution found!
s: 9
e: 5
n: 6
d: 7
m: 1
o: 0
r: 8
y: 2


In [11]:
9567 + \
1085

10652

# Sudoku

In [3]:
def solve_sudoku():
    """Solves the sudoku problem with the CP-SAT solver."""
    # Create the model.
    model = cp_model.CpModel()

    cell_size = 3
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    initial_grid = [
        [0, 6, 0, 0, 5, 0, 0, 2, 0],
        [0, 0, 0, 3, 0, 0, 0, 9, 0],
        [7, 0, 0, 6, 0, 0, 0, 1, 0],
        [0, 0, 6, 0, 3, 0, 4, 0, 0],
        [0, 0, 4, 0, 7, 0, 1, 0, 0],
        [0, 0, 5, 0, 9, 0, 8, 0, 0],
        [0, 4, 0, 0, 0, 1, 0, 0, 6],
        [0, 3, 0, 0, 0, 8, 0, 0, 0],
        [0, 2, 0, 0, 4, 0, 0, 5, 0],
    ]

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.NewIntVar(1, line_size, "grid %i %i" % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.AddAllDifferent(grid[(i, j)] for j in line)

    # AllDifferent on columns.
    for j in line:
        model.AddAllDifferent(grid[(i, j)] for i in line)

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di, j * cell_size + dj)])

            model.AddAllDifferent(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.Add(grid[(i, j)] == initial_grid[i][j])

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        for i in line:
            print([int(solver.Value(grid[(i, j)])) for j in line])


solve_sudoku()

[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]


# N-Queens

In [4]:
def solve_nqueens(N):
    """Solve the N-Queens problem."""
    # Create the model.
    model = cp_model.CpModel()

    # There are `N` number of variables, one for a queen in each column
    # of the board. The value of each variable is the row that the queen is in.

    queens = [model.NewIntVar(0, N - 1, f"x_{i}") for i in range(N)]

    # All rows must be different.
    model.AddAllDifferent(queens)

    # No two queens can be on the same diagonal.
    model.AddAllDifferent(queens[i] + i for i in range(N))
    model.AddAllDifferent(queens[i] - i for i in range(N));

    solver = cp_model.CpSolver()
    solver.parameters.enumerate_all_solutions = True
    solution_printer = cp_model.ObjectiveSolutionPrinter()
    
    print(f"NQueens(N={N})")
    status = solver.Solve(model, solution_printer)

    # Print solution.
    print(f"status: {solver.StatusName(status)}")
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        for idx, var in enumerate(queens):
            print(f"Q[{idx}]: {solver.Value(var)}")


In [5]:
solve_nqueens(5)

NQueens(N=5)
Solution 0, time = 0.01 s, objective = 0
Solution 1, time = 0.01 s, objective = 0
Solution 2, time = 0.01 s, objective = 0
Solution 3, time = 0.01 s, objective = 0
Solution 4, time = 0.01 s, objective = 0
Solution 5, time = 0.01 s, objective = 0
Solution 6, time = 0.01 s, objective = 0
Solution 7, time = 0.01 s, objective = 0
Solution 8, time = 0.01 s, objective = 0
Solution 9, time = 0.01 s, objective = 0
status: OPTIMAL
Q[0]: 0
Q[1]: 2
Q[2]: 4
Q[3]: 1
Q[4]: 3


# Golumb Ruler

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Golomb_Ruler-4.svg/440px-Golomb_Ruler-4.svg.png)

In [6]:
def solve_golomb_ruler(order, params=None):
    """Solve the Golomb ruler problem."""
    # Create the model.
    model = cp_model.CpModel()

    var_max = order * order

    all_vars = list(range(0, order))

    marks = [model.NewIntVar(0, var_max, f"marks_{i}") for i in all_vars]

    model.Add(marks[0] == 0)
    for i in range(order - 2):
        model.Add(marks[i + 1] > marks[i])

    diffs = []
    for i in range(order - 1):
        for j in range(i + 1, order):
            diff = model.NewIntVar(0, var_max, f"diff [{j},{i}]")
            model.Add(diff == marks[j] - marks[i])
            diffs.append(diff)
    model.AddAllDifferent(diffs)

    # symmetry breaking
    if order > 2:
        model.Add(marks[order - 1] - marks[order - 2] > marks[1] - marks[0])

    # Objective
    model.Minimize(marks[order - 1])

    # Solve the model.
    solver = cp_model.CpSolver()
    if params:
        text_format.Parse(params, solver.parameters)
    solution_printer = cp_model.ObjectiveSolutionPrinter()
    print(f"Golomb ruler(order={order})")
    status = solver.Solve(model, solution_printer)

    # Print solution.
    print(f"status: {solver.StatusName(status)}")
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        for idx, var in enumerate(marks):
            print(f"mark[{idx}]: {solver.Value(var)}")
        intervals = [solver.Value(diff) for diff in diffs]
        intervals.sort()
        print(f"intervals: {intervals}")

    print("Statistics:")
    print(f"- conflicts: {solver.NumConflicts()}")
    print(f"- branches : {solver.NumBranches()}")
    print(f"- wall time: {solver.WallTime()}s\n")

In [7]:
solve_golomb_ruler(order = 4)

Golomb ruler(order=4)
Solution 0, time = 0.02 s, objective = 7
Solution 1, time = 0.03 s, objective = 6
status: OPTIMAL
mark[0]: 0
mark[1]: 1
mark[2]: 4
mark[3]: 6
intervals: [1, 2, 3, 4, 5, 6]
Statistics:
- conflicts: 0
- branches : 0
- wall time: 0.0396911s



In [8]:
solve_golomb_ruler(order = 6)

Golomb ruler(order=6)
Solution 0, time = 0.01 s, objective = 20
Solution 1, time = 0.01 s, objective = 18
Solution 2, time = 0.01 s, objective = 17
status: OPTIMAL
mark[0]: 0
mark[1]: 1
mark[2]: 4
mark[3]: 10
mark[4]: 12
mark[5]: 17
intervals: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17]
Statistics:
- conflicts: 0
- branches : 0
- wall time: 0.017013s



# Map Coloring

![](https://dmcommunity.files.wordpress.com/2019/05/mapcoloring.png)

In [9]:
def map_color():

  model = cp_model.CpModel()

  #
  # data
  #
  Belgium = 0
  Denmark = 1
  France = 2
  Germany = 3
  Netherlands = 4
  Luxembourg = 5

  n = 6
  max_num_colors = 4

  # declare variables
  color = [model.NewIntVar(1, max_num_colors, f"x_{i}") for i in range(n)]

  #
  # constraints
  #
  model.Add(color[Belgium] == 1)  # Symmetry breaking
  model.Add(color[France] != color[Belgium])
  model.Add(color[France] != color[Luxembourg])
  model.Add(color[France] != color[Germany])
  model.Add(color[Luxembourg] != color[Germany])
  model.Add(color[Luxembourg] != color[Belgium])
  model.Add(color[Belgium] != color[Netherlands])
  model.Add(color[Belgium] != color[Germany])
  model.Add(color[Germany] != color[Netherlands])
  model.Add(color[Germany] != color[Denmark])

  #
  # solution and search
  #
  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter(color)
  status = solver.SearchForAllSolutions(model, solution_printer)

  if not (status == cp_model.OPTIMAL or status == cp_model.FEASIBLE):
    print("No solutions found")

  print()
  print("NumSolutions:", solution_printer.solution_count())
  print("NumConflicts:", solver.NumConflicts())
  print("NumBranches:", solver.NumBranches())
  print("WallTime:", solver.WallTime())

In [10]:
map_color()

NameError: name 'VarArraySolutionPrinter' is not defined

# Lecture Scheduling


  Suppose we wish to schedule six one-hour lectures, v1, v2, v3, v4, v5, v6.
  
  Among the the potential audience there are people who wish to hear both

   - v1 and v2
   - v1 and v4
   - v3 and v5
   - v2 and v6
   - v4 and v5
   - v5 and v6
   - v1 and v6

  How many hours are necessary in order that the lectures can be given
  without clashes?
  

In [ ]:
def lecture_schedule():

  model = cp_model.CpModel()

  #
  # data
  #

  #
  # The schedule requirements:
  # lecture a cannot be held at the same time as b
  # Note: 1-based
    
  g = [[1, 2], [1, 4], [3, 5], [2, 6], [4, 5], [5, 6], [1, 6]]

  # number of nodes
  n = 6

  # number of edges
  edges = len(g)

  #
  # declare variables
  #
    
  v = [model.NewIntVar(0, n - 1, 'v[%i]' % i) for i in range(n)]

  # maximum color, to minimize
  # Note: since Python is 0-based, the
  # number of colors is +1
  max_c = model.NewIntVar(0, n - 1, 'max_c')

  #
  # constraints
  #
  model.AddMaxEquality(max_c, v)

  # ensure that there are no clashes
  # also, adjust to 0-base
  for i in range(edges):
    model.Add(v[g[i][0] - 1] != v[g[i][1] - 1])

  # symmetry breaking:
  # - v0 has the color 0,
  # - v1 has either color 0 or 1
    
  model.Add(v[0] == 0)
  model.Add(v[1] <= 1)

  # objective
  model.Minimize(max_c)

  #
  # solution and search
  #
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  if status == cp_model.OPTIMAL:
    print("max_c:", solver.Value(max_c) + 1, "colors")
    print("v:", [solver.Value(v[i]) for i in range(n)])
    print()

  print("NumConflicts:", solver.NumConflicts())
  print("NumBranches:", solver.NumBranches())
  print("WallTime:", solver.WallTime())

lecture_schedule()

max_c: 3 colors
v: [0, 1, 0, 2, 1, 2]

NumConflicts: 0
NumBranches: 0
WallTime: 0.004514000000000001


# Coins Grid

 - In each row exactly c coins must be placed.
 - In each column exactly c coins must be placed.
 - The sum of the quadratic horizontal distance from the main diagonal of all cells containing a coin must be as small as possible.
 - In each cell at most one coin can be placed.

In [ ]:
def coins_grid(n=31, c=14):
    
  model = cp_model.CpModel() 

  # data
  print("n: ", n)
  print("c: ", c)

  # declare variables
  x = {}
  for i in range(n):
    for j in range(n):
      x[(i, j)] = model.NewBoolVar(f"x {i} {j}")

  #
  # constraints
  #

  # sum rows/columns == c
    
  for i in range(n):
    model.Add(sum([x[(i, j)] for j in range(n)]) == c)  # sum rows
    model.Add(sum([x[(j, i)] for j in range(n)]) == c)  # sum cols

  # quadratic horizonal distance var
    
  obj = model.NewIntVar(0, n*n*c*c, "obj") 
  model.Add(obj == sum([x[(i, j)] * (i - j) * (i - j) for i in range(n) for j in range(n)]))

  # objective
  model.Minimize(obj)

  # Search and solution
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  if status == cp_model.OPTIMAL:
    print("obj:", solver.Value(obj))
    for i in range(n):
      for j in range(n):
        print(solver.Value(x[(i, j)]), end=" ")
      print()
    print()

  print("NumConflicts:", solver.NumConflicts())
  print("NumBranches:", solver.NumBranches())
  print("WallTime:", solver.WallTime())

In [ ]:
coins_grid(n = 8, c = 4)

n:  8
c:  4
obj: 80
1 1 1 1 0 0 0 0 
1 1 1 1 0 0 0 0 
1 1 1 1 0 0 0 0 
1 1 1 1 0 0 0 0 
0 0 0 0 1 1 1 1 
0 0 0 0 1 1 1 1 
0 0 0 0 1 1 1 1 
0 0 0 0 1 1 1 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.012405000000000001


In [ ]:
coins_grid(n = 12, c = 5)

n:  12
c:  5
obj: 234
1 1 1 1 1 0 0 0 0 0 0 0 
1 1 1 1 1 0 0 0 0 0 0 0 
1 1 1 1 1 0 0 0 0 0 0 0 
1 1 1 1 0 1 0 0 0 0 0 0 
1 1 1 0 0 1 1 0 0 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 0 
0 0 0 0 1 1 1 1 1 0 0 0 
0 0 0 0 0 1 1 0 0 1 1 1 
0 0 0 0 0 0 1 0 1 1 1 1 
0 0 0 0 0 0 0 1 1 1 1 1 
0 0 0 0 0 0 0 1 1 1 1 1 
0 0 0 0 0 0 0 1 1 1 1 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.049262


In [ ]:
coins_grid(n = 31, c = 14)

n:  31
c:  14
obj: 13668
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 